<a href="https://colab.research.google.com/github/MelKanyi/Week-6-IP/blob/master/Moringa_Data_Science_Core_W6_Independent_Project_2020_Melissa_Kanyi_Python_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Defining the Question

### a) Specifying the Question

> Predict the result of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the game is friendly - using five features i.e Home team, Away team, Tournament type, Rank of home team and Rank of away team


### b) Defining the Metric for Success
Predicting, with an accuracy of 70% or an RMSE of 10% of the target variable mean
 
         1: how many goals the home team scores

         2: how many goals the away team scores

### c) Understanding the context 
My context will be in the form of the Project Overview

        You have been recruited as a football analyst in a company - Mchezopesa Ltd and tasked to accomplish the task below.

          A Prediction result of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the 
          game is friendly (include rank in your training).

          You have two possible approaches (as  shown below) given the datasets that will be provided

          Input: Home team, Away team, Tournament type (World cup, Friendly, Other)

          Approach 1: Polynomial approach

          What to train given:

          Rank of home team
          Rank of away team
          Tournament type
          Model 1: Predict how many goals the home team scores

          Model 2: Predict how many goals the away team scores

 

          Approach 2 Logistic approach

          Feature Engineering: Figure out from the home team’s perspective if the game is a Win, Lose or Draw (W, L, D)

### d) Recording the Experimental Design
The expected flow for the assessment will be as follows:

    - Perform your EDA
    - Perform any necessary feature engineering
    - Check of multicollinearity
    - Start building the model
    - Cross-validate the model
    - Compute RMSE
    - Create residual plots for your models, and assess their heteroscedasticity using Bartlett’s test
    - Perform appropriate regressions on the data including your justification
    
        NB: The assessment might not discretely follow the flow as outlined

# **Import necessary Libraries**

In [0]:
# Lets load all the necessary libraries for our workflow.
#
# Analysis libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy as sp
import pandas_profiling as pp

# Machine learning libraries

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, confusion_matrix
from scipy import stats
from scipy.stats import ttest_ind
from statsmodels.stats import weightstats as stests
import warnings
warnings.filterwarnings('ignore')

# Other libraries
import datetime as dt

# 2. Reading the Data

In [0]:
# reading the data and changing it to datetime 
ranks = pd.read_csv('/content/fifa_ranking.csv', parse_dates = ['rank_date'])
ranks

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07


In [0]:
# reading the data and changing it to datetime 
results = pd.read_csv('/content/results.csv', parse_dates = ['date'])
results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True


# 3. Checking the Data

In [0]:
# Determining the no. of records in our dataset
#

In [0]:
# Previewing the top of our dataset
#
ranks.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


In [0]:
# Previewing the bottom of our dataset
# 
ranks.tail()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57792,206,Tonga,TGA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OFC,2018-06-07


In [0]:
# Previewing the top of our dataset
#
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [0]:
# Previewing the bottom of our dataset
# 
results.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True
40838,2019-07-20,Papua New Guinea,Fiji,1,1,Pacific Games,Apia,Samoa,True


In [0]:
# Checking whether each column has an appropriate datatype
#
ranks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
rank                       57793 non-null int64
country_full               57793 non-null object
country_abrv               57793 non-null object
total_points               57793 non-null float64
previous_points            57793 non-null int64
rank_change                57793 non-null int64
cur_year_avg               57793 non-null float64
cur_year_avg_weighted      57793 non-null float64
last_year_avg              57793 non-null float64
last_year_avg_weighted     57793 non-null float64
two_year_ago_avg           57793 non-null float64
two_year_ago_weighted      57793 non-null float64
three_year_ago_avg         57793 non-null float64
three_year_ago_weighted    57793 non-null float64
confederation              57793 non-null object
rank_date                  57793 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(9), int64(3), object(3)
memory usage: 7.1+ MB


In [0]:
# Checking whether each column has an appropriate datatype
#
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40839 entries, 0 to 40838
Data columns (total 9 columns):
date          40839 non-null datetime64[ns]
home_team     40839 non-null object
away_team     40839 non-null object
home_score    40839 non-null int64
away_score    40839 non-null int64
tournament    40839 non-null object
city          40839 non-null object
country       40839 non-null object
neutral       40839 non-null bool
dtypes: bool(1), datetime64[ns](1), int64(2), object(5)
memory usage: 2.5+ MB


# Working on our results dataset


In [0]:
# subset our data to start from 1993-08-08
res= results[results['date']>= '1993-08-08'] 
res.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
17915,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False
17916,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False
17917,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False
17918,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False
17919,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False


In [0]:
# we are going to extract the year value from the date column the impute the year values in a new column 
def extract_date(results, column):
    results["year"] = results[column].apply(lambda x: x.year)
    
extract_date(res, 'date')
res.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
17915,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,1993
17916,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False,1993
17917,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False,1993
17918,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False,1993
17919,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,1993


In [0]:
# we are going to extract the year value from the date column the impute the year values in a new column 
def extract_date(ranks, column):
    ranks["year"] = ranks[column].apply(lambda x: x.year)
    
extract_date(ranks, 'rank_date')
ranks.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date,year
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08,1993
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08,1993
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08,1993
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08,1993
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08,1993


# 5. Tidying the Dataset

In [0]:
# dropping unnecesarry columns from the ranks dataset 

ranks.drop(ranks.columns.difference(['rank','country_full' , 'year']), 1, inplace=True)
ranks

,rank,country_full,year
0,1,Germany,1993
1,2,Italy,1993
2,3,Switzerland,1993
3,4,Sweden,1993
4,5,Argentina,1993
...,...,...,...
57788,206,Anguilla,2018
57789,206,Bahamas,2018
57790,206,Eritrea,2018
57791,206,Somalia,2018


In [0]:
res.head(1)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
17915,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,1993


In [0]:
# dropping unnecesarry columns from the res dataset 

res.drop(res.columns.difference(['home_team','away_team','home_score','away_score','tournament','country','neutral','year']), 1, inplace=True)
res

,home_team,away_team,home_score,away_score,tournament,country,neutral,year
17915,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,False,1993
17916,Brazil,Mexico,1,1,Friendly,Brazil,False,1993
17917,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Ecuador,False,1993
17918,Guinea,Sierra Leone,1,0,Friendly,Guinea,False,1993
17919,Paraguay,Argentina,1,3,FIFA World Cup qualification,Paraguay,False,1993
...,...,...,...,...,...,...,...,...
40834,American Samoa,Tahiti,8,1,Pacific Games,Samoa,True,2019
40835,Fiji,Solomon Islands,4,4,Pacific Games,Samoa,True,2019
40836,Senegal,Algeria,0,1,African Cup of Nations,Egypt,True,2019
40837,Tajikistan,North Korea,0,1,Intercontinental Cup,India,True,2019


In [0]:
# identify missing data in the ranks dataset
ranks.isnull().any()

rank            False
country_full    False
year            False
dtype: bool

In [0]:
# identify missing values in the res dataset
res.isnull().any()

home_team     False
away_team     False
home_score    False
away_score    False
tournament    False
country       False
neutral       False
year          False
dtype: bool

> From the codes above, we can see that there are no missing values in both dataframes

In [0]:
# creating a new dataframe with rows with neutral false only
res_neutral = res[res['neutral'] == False]
res_neutral.head()

,home_team,away_team,home_score,away_score,tournament,country,neutral,year
17915,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,False,1993
17916,Brazil,Mexico,1,1,Friendly,Brazil,False,1993
17917,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Ecuador,False,1993
17918,Guinea,Sierra Leone,1,0,Friendly,Guinea,False,1993
17919,Paraguay,Argentina,1,3,FIFA World Cup qualification,Paraguay,False,1993


In [0]:
# drop the neutral column

res_neutral.drop(res.columns.difference(['home_team','away_team','home_score','away_score','tournament','country','year']), 1, inplace=True)
res_neutral

,home_team,away_team,home_score,away_score,tournament,country,year
17915,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,1993
17916,Brazil,Mexico,1,1,Friendly,Brazil,1993
17917,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Ecuador,1993
17918,Guinea,Sierra Leone,1,0,Friendly,Guinea,1993
17919,Paraguay,Argentina,1,3,FIFA World Cup qualification,Paraguay,1993
...,...,...,...,...,...,...,...
40803,Samoa,Papua New Guinea,0,6,Pacific Games,Samoa,2019
40817,Samoa,Tonga,2,0,Pacific Games,Samoa,2019
40821,India,North Korea,2,5,Intercontinental Cup,India,2019
40829,India,Syria,1,1,Intercontinental Cup,India,2019


In [0]:
# merging 
new_df = pd.merge(ranks, res_neutral,  how='inner', left_on=['country_full','year'], right_on = ['home_team','year'] )
new_df.head(60)

,rank,country_full,year,home_team,away_team,home_score,away_score,tournament,country
0,1,Germany,1993,Germany,Uruguay,5,0,Friendly,Germany
1,1,Germany,1993,Germany,Brazil,2,1,Friendly,Germany
2,5,Germany,1993,Germany,Uruguay,5,0,Friendly,Germany
3,5,Germany,1993,Germany,Brazil,2,1,Friendly,Germany
4,4,Germany,1993,Germany,Uruguay,5,0,Friendly,Germany
5,4,Germany,1993,Germany,Brazil,2,1,Friendly,Germany
6,3,Germany,1993,Germany,Uruguay,5,0,Friendly,Germany
7,3,Germany,1993,Germany,Brazil,2,1,Friendly,Germany
8,1,Germany,1993,Germany,Uruguay,5,0,Friendly,Germany
9,1,Germany,1993,Germany,Brazil,2,1,Friendly,Germany


# 6. Exploratory Data Analysis

In [0]:
# We will use Pandas Profiling to get an overview of both our dataframes and their statistical summaries
# Getting an overview of the results dataframe
pp.ProfileReport(results)

In [0]:
# Getting an overview of the ranking dataframe
pp.ProfileReport(ranking)

In [0]:
# Converting the date data type in the results dataframe from object to datetime
# 
results.date = pd.to_datetime(results.date)

In [0]:
# Let's convert the date column dtype in the ranking dataframe from object to datetime
# 
ranking.rank_date = pd.to_datetime(ranking.rank_date)

# 7. Implementing the Solution

# 8. Challenging the solution

> The easy solution is nice because it is, well, easy, but you should never allow those results to hold the day. You should always be thinking of ways to challenge the results, especially if those results comport with your prior expectation.


In [0]:
# Reviewing the Solution 
#

## 1.0 Connecting to our Database

In [0]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [0]:
# Loading our table from the respective CSV files
# We will export the dataset that we have cleaned using python
#
with open('results.csv','r') as f:
    results_sql = pd.read_csv(f, index_col=0, encoding='utf-8')
    
%sql DROP TABLE if EXISTS results_sql;

%sql PERSIST results_sql;

%sql SELECT * FROM results_sql LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,0
1873-03-08,England,Scotland,4,2,Friendly,London,England,0
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,0
1875-03-06,England,Scotland,2,2,Friendly,London,England,0
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,0


In [0]:
# Loading our table from the respective CSV files
# We will export the dataset that we have cleaned using python
#
with open('fifa_ranking.csv','r') as f:
    ranks_sql = pd.read_csv(f, index_col=0, encoding='utf-8')
    
%sql DROP TABLE if EXISTS ranks_sql;

%sql PERSIST ranks_sql;

%sql SELECT * FROM ranks_sql LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
